In [ ]:
# IMPORTS
import pydicom
import matplotlib.pyplot as plt
import numpy as np
import copy
from scipy.ndimage import convolve
import cv2
import nibabel as nib
from sklearn.cluster import KMeans, DBSCAN

La première partie du travail consiste en l'application de méthodes de traitement d'image afin d'améliorer la qualité des images. Pour ce faire, nous utilisons des fichiers dicoms que l'on va traiter en Numpy arrays. En ce qui concerne l'implémentation des fonctions, nous avons essayé de toutes les implémenter nous-mêmes. Cependant, ceci n'étant pas toujours très efficace, nous préférons utiliser des méthodes toutes faites fournies par les librairies Python. L'ensemble de ces fonctions demandent un Numpy array en input, c'est-à-dire un array de pixels. 

Globalement, pour améliorer la qualité des images, nous avons appliqué un linear strech, une égalisation ainsi que l'algorithme CLAHE.

In [ ]:
#Retourne l'histogramme normalisé de l'array 
#En normalisant l'histogramme, nous le trouvions plus lisible
def histogram(arr):
    maxValue = np.max(arr)
    h,w = arr.shape
    n = h*w
    histogram = np.array([0]*(maxValue+1))
    for i in arr:
        for j in i:
            histogram[j] += 1
    histogram_n = histogram/n#histogramme normalisé
    return histogram_n

#Retourne l'histogramme cumulé de l'array
def cumulative(arr):
    maxValue = np.max(arr)
    cumul = np.zeros((maxValue))
    histo = histogram(arr)
    for i in range(len(cumul)):
        cumul[i] = np.sum(histo[0:i+1])
    return cumul
# This function is transforming an image with the Transform vector --> Je sais pas ce que fait cette fonction
def Transformation(image, transform):
    newArr = copy.deepcopy(image)
    for i in range(len(image)):
        for j in range(len(image[i])):
            newArr[i, j] = transform[image[i, j]]
    return newArr  

#Réalise une interpolation linéaire pour le linear strech de l'histogramme
#Dans notre cas, les images sont de taille 260x320 et sont encodées sur 345 valeurs. Nous souhaitons faire une interpolation
#sur 256 valeurs et donc, nous réalisons une compression et non pas une élongation de l'histogramme.
def crimping(arr):
    maxValue = np.max(arr)
    histo = histogram(arr)
    Transform = np.arange(0,  maxValue + 1)
    deltaA = (maxValue - np.min(np.where(histo > 0)))/(255)
    Transform = np.ceil(Transform / deltaA)   
    newArr = Transformation(arr, Transform)
    return newArr

#Réalise une égalisation de l'histogramme de facon à ce qu'il n'y ait pas une valeur de pixel qui soit plus utilisée
def equalize(arr):
    h,w = arr.shape
    n = h*w
    C = ((344-1)/n)
    new_arr = np.zeros((h,w),dtype=int)
    histo = histogram(arr)*n
    for i in range(h):
        for j in range(w):
            value = arr[i,j]
            new_arr[i,j] = C*(np.sum(histo[0:value+1]))
    return new_arr

#Notre implémentation de CLAHE: pour mémoire car il prend trop de temps
#cet algorithme consiste à appliquer une égalisation d'histogramme sur des blocs de pixels de taille 8x8 (en général). 
#Il permet donc de ne pas amplifier le bruit.
def Clahe(arr):
    h,w = arr.shape
    new_arr = np.zeros((h,w))
    for i in range(h):
        for j in range(w):
            C = arr[i:i+7,j:j+7]
            new_C = equalize(C)
            new_arr[i:i+7,j:j+7] = new_C
            if (j+7 >= w):
                break
            else:
                j = j+7
        if(i+7>=h):
            break
        else:
            i = i+7
    plt.imshow(new_arr, cmap='Greys')

#Implémentation de python
def clahe(arr):
    clahe = cv2.createCLAHE()
    cl1 = clahe.apply(arr)
    return cl1

Nous allons maintenant afficher les résultats obtenus sur une même image mais avec les différentes méthodes.

In [ ]:
#calcul des variables utiles
arr_crimp = crimping(arr)
histo = histogram(arr)#histogramme de l image non traitee
histogram_lin = histogram(arr_crimp)#histogramme de l image apres crimping

arr_egal = equalize(arr)
histogram_egal = histogram(arr_egal)#histogramme de l image apres égalisation

cl1 = clahe(arr)
histo = histogram(arr)#histogramme de l image non traitee
histo_cl1 = histogram(cl1)#histogramme après CLAHE

cumul = cumulative(arr)
cumul_lin = cumulative(arr_crimp)
cumul_egal = cumulative(arr_egal)

#plot des figures relatives au crimping
plt.figure(figsize = (15,10))
plt.subplot(2,2,1)
plt.imshow(arr, cmap='Greys')
plt.title('Image non traitée')

plt.subplot(2,2,2)
plt.imshow(arr_crimp,cmap='Greys')
plt.title('Image après crimping')

plt.subplot(2,2,3)
plt.plot(histo)
plt.title('Histogramme de l\'image non traitée')

plt.subplot(2,2,4)
plt.plot(histogram_lin)
plt.title('Histogramme de l\'image apres crimping')

#plot des figures relatives à l'égalisation des histogrammes
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.imshow(arr, cmap='Greys')
plt.title('Image non traitée')

plt.subplot(2,2,2)
plt.imshow(arr_egal,cmap='Greys')
plt.title('Image après égalisation de l\'histogramme')

plt.subplot(2,2,3)
plt.plot(histo)
plt.title('Histogramme de l\'image non traitée')

plt.subplot(2,2,4)
plt.plot(histogram_egal)
plt.title('Histogramme de l\'image apres égalisation')

#plot des figures relatives aux histogrammes cumulés
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.plot(cumul)
plt.title('Histogramme cumulé de l\'image non traitée')
          
plt.subplot(1,3,2)
plt.plot(cumul_lin)
plt.title('Histogramme cumulé de l\'image après crimping')

plt.subplot(1,3,3)
plt.plot(cumul_egal)
plt.title('Histogramme cumulé de l\'image après égalisation')

#Plot des résultats avec l'algorithme CLAHE
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.imshow(arr, cmap='Greys')
plt.title('Image non traitée')

plt.subplot(2,2,2)
plt.imshow(cl1, cmap='Greys')
plt.title('Image traitée avec l algorithme CLAHE')

plt.subplot(2,2,3)
plt.plot(histo)
plt.title('Histogramme de l image non traitée')

plt.subplot(2,2,4)
plt.plot(histo_cl1)
plt.title('Histogramme de l image traitée avec CLAHE')